In [1]:
import sys
sys.path.append('../')
import time
from datasets import load_dataset
import transformers
transformers.logging.set_verbosity_warning()
from transformers import AutoTokenizer
from src.frameworks import TruncatedMC

c:\Users\AronQ\.conda\envs\flenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (C:/Users/AronQ/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00,  4.26it/s]


In [3]:
dataset["train"][2]

{'label': 3,
 'text': "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-aad1af4c7095bfa1.arrow
Loading cached processed dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-29f27748f0b54d01.arrow


In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(5))

Loading cached shuffled indices for dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-11a7619c6a3c070f.arrow
Loading cached shuffled indices for dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-3c5c2a245be1b332.arrow


In [8]:
type(small_train_dataset)

datasets.arrow_dataset.Dataset

In [9]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})

In [10]:
type(small_train_dataset['text'])

list

The following cell output should contain a warning like: "Some weights of the model checkpoint at bert-base-cased were not used when initializing...".

This is OK as we are not using the pooler layer in order to compute the loss.

In [11]:
dshap = TruncatedMC(train_dataset=small_train_dataset, X_train=small_train_dataset['text'], X_test=small_eval_dataset['text'], test_dataset=small_eval_dataset, model_family='bert-base-cased', num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Print the initial accuracy

In [12]:
dshap.random_score

0.4

In [13]:
time.asctime()

'Tue May  2 18:31:51 2023'

In [14]:
dshap.run(save_every=100, err=0.1, do_loo=True, do_tmc=False, do_gshap=False)

Calculate leave-one-out


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

{'train_runtime': 16.064, 'train_samples_per_second': 0.623, 'train_steps_per_second': 0.125, 'train_loss': 1.646740436553955, 'epoch': 2.0}
{'eval_loss': 1.9394981861114502, 'eval_accuracy': 0.4, 'eval_runtime': 3.219, 'eval_samples_per_second': 1.553, 'eval_steps_per_second': 0.311, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

{'train_runtime': 13.1141, 'train_samples_per_second': 0.61, 'train_steps_per_second': 0.153, 'train_loss': 1.337770700454712, 'epoch': 2.0}


...

{'eval_loss': 2.0160582065582275, 'eval_accuracy': 0.6, 'eval_runtime': 3.402, 'eval_samples_per_second': 1.47, 'eval_steps_per_second': 0.294, 'epoch': 2.0}


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

{'train_runtime': 13.683, 'train_samples_per_second': 0.585, 'train_steps_per_second': 0.146, 'train_loss': 1.4319584369659424, 'epoch': 2.0}


100%|██████████| 5/5 [01:30<00:00, 18.05s/it]

{'eval_loss': 2.1640307903289795, 'eval_accuracy': 0.2, 'eval_runtime': 3.452, 'eval_samples_per_second': 1.448, 'eval_steps_per_second': 0.29, 'epoch': 2.0}


In [15]:
time.asctime()

'Tue May  2 18:33:42 2023'

In [16]:
dshap.vals_loo

array([ 0.2,  0. ,  0.2, -0.2,  0.2])